# Laboratoire 3 - Caméra
Matériel fourni: règle en aluminium de 1 m, règle en plastique de 30 cm, rapporteur d'angle, repère vertical.

## Partie 0 - Interagir avec la caméra

D'abord, exécutons un peu de code de configuration, puis connectons-nous au robot.

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import numpy.linalg as lin

from robmob.robot import Robot
from robmob.rover.sensors import CameraRGBSensor

%matplotlib ipympl

In [ ]:
ip_robot = 'localhost'
robot = Robot(ip_robot)
robot.connect()

Le code suivant sert à créer ajouter la caméra aux capteurs du robot.

In [ ]:
camera = CameraRGBSensor()
robot.add_sensor(camera)

> **PROTIP** Aussitôt que vous exécutez la ligne `robot.add_sensor(camera)`, le flux de la caméra est transmi 
> par le réseau du robot jusqu'à votre ordinateur. Comme ces données causent un bon traffic sur le routeur, 
> tentez de ne pas vous connecter à ce flux de données plusieurs fois en parallèle!


## Acquisition d'images

Pour afficher la dernière image capturée par la caméra, on peut utiliser `peek_data`.

In [ ]:
# img = camera.peek_data()

img = cv2.imread('offline/Labo 3/focal.png')
plt.imshow(img[:, :, ::-1])
plt.show()

## Partie 1 - Calibration de la longueur focale

L'objectif de cette partie de de déterminer la longueur focale de la caméra du robot. Nous utiliserons la caméra RGB, en ignorant pour l'instant la partie infra-rouge des données.

> **NOTE** La OAK possède plusieurs caméras, validez laquelle génère l'image RGB pour vos expérimentations. Mettez cette caméra au dessus du 0 de la règle d'un mètre.

Pour commencer, placez la règle de 1 m sur une table, puis placez le robot à une des extrémités de la règle. À l'autre extrémité, placez la règle de plastique de 30 cm à la même hauteur que la caméra, de sorte que la caméra observe directement la règle de plastique. La caméra devrait être à environ 50 cm de la règle de plastique. Vous pouvez utiliser une feuille de papier derrière la règle de plastique pour rendre les mesures plus faciles à prendre. Votre assemblage devrait ressembler à l'image ci-bas.

![Assemblage pour la calibration](img/assemblage_calibration.jpg)

La commande ci-bas va afficher une image prise par la caméra dans une console interactive. Quand vous passez votre curseur de souris dans la console interactive, on devrait vous indiquer à quelle position, en pixel, se trouve votre curseur. Prenez en note la position de deux pixels qui sont à une distance connue. Par exemple, vous pourriez prendre en note la position de deux pixels qui sont à 5 cm de distance sur la règle. Pour vous faciliter la tâche, n'hésitez pas à utiliser le bouton *zoom to rectangle* de la console interactive, qui vous permettera de zoomer sur la règle et de voir les chiffres plus facilement.

> **ATTENTION** Dans le présent *jupyter notebook*, il est important de fermer les figures (en appuyant sur le bouton bleu) quand vous avez terminé des les consulter. Jupyter ne semble pas capable d'afficher deux figures en même temps.

In [ ]:
# La boite de PLA fait 19.5 cm de large et est a une distance de 50cm
img = cv2.imread('offline/Labo 3/focal.png')
plt.imshow(img[:, :, ::-1])
plt.show()

Utilisez le théorème de Thalès pour déduire la longueur focale de la caméra. En guise de rappel, la longueur focale est donnée par 

$$ f = \Delta L_{caméra} \frac{A_z}{\Delta L_{réel}} $$

In [ ]:
first_pixel_x = 285
second_pixel_x = 335

delta_l_camera = second_pixel_x - first_pixel_x
delta_l_real_object = ...  # TODO Distance réelle entre les deux pixels sélectionnés
distance_to_ruler = ...  # TODO distance entre la camera et le pilier

f = ...  # TODO calculer la longueur focale
print('Longueur focale: {}'.format(f))

Pour valider vos calculs, déplacez la règle de plastique à une distance différente de la caméra. Toujours en utilisant le théorème de Thalès, estimez la distance $A_z$ en utilisant la longueur focale que vous avez trouvé. Votre estimation de $A_z$ et sa valeur réelle devraient être similaire.

## Partie 2 - Mesure d'angles

Comme mentionné en classe, la caméra est un rapporteur d'angle. Cette partie décrit comment faire des mesures d'angles avec la caméra du robot.

Placez deux objets dans le champ de vision de la caméra. En vous servant des règles et d'un rapporteur d'angle, mesurez l'angle approximatif entre ces deux objets, du point de vue de la caméra.

À l'aide de la commande suivante, trouvez les coordonnées en $x$ du centre de chacun des objets.

**Important:** Pour les données de `offline`, considérez le centre de la tasse comme étant le centre de l'objet.

In [ ]:
# Essayez avec local_1.png
# Considerez le centre des tasses
img = cv2.imread('offline/Labo 3/local_1.png')
plt.imshow(img[::-1, :, ::-1])
plt.show()

In [ ]:
p1 = 0
p2 = 0

Calculez l'angle $\theta$ entre les deux objets. Pour ce faire, vous devez considérer la distance en x (sur le plan image) entre le centre de l'objet et le centre optique de la caméra. Le centre optique passe au milieu de l'image, la colonne `1920 // 2` dans notre cas. En guise de rappel, voici un schéma qui pourrait vous aider à calculer les angles nécessaires.

![](img/calcul_angle.png)

> **PROTIP** La fonction `arctan2` de numpy pourrait vous être utile! Comme nous avons renommé `numpy` pour `np` dans le haut du document, vous pouvez l'appeler en faisant `np.arctan2()`.

In [ ]:
lx1 = 0  # TODO
theta1 = 0  # TODO

lx2 = 0  # TODO
theta2 = 0  # TODO

angle_between_objects = 0  # TODO
print(angle_between_objects)

## Partie 3 - Localisation en deux dimensions

Placez cette fois trois objets sur le plancher, en vous servant des tuiles comme système cartésien. Le point de repère `p2` sera la position (0,0) de votre référentiel monde. Ce système de coordonnées aura comme unité de longueur une *tuile*, soit environ 30 cm. Placez deux autres objets aux intersections de tuiles, de sorte à avoir le montage ci-bas.

![img](img/montage_crayon.png)

Avec le code suivant, capturez une image, puis notez la position en $x$ de chaque objet. Calculez l'angle $\alpha$ entre les objets 1 et 2 puis l'angle $\beta$ entre les objets 2 et 3. 

**Important:** Consultez les notes de cours `03-Vision-B.pdf` pour comprendre comment calculer ces angles. L'algorithme présenté dans `circle_from_pts_and_angle` sera vu plus en détail dans le cours magistral.

In [ ]:
plt.imshow(camera.peek_data())

In [ ]:
def distance(p1, p2):
    return np.sqrt((p2[0, 0] - p1[0, 0]) ** 2 + (p2[1, 0] - p1[1, 0]) ** 2)


# TODO completer cette fonction
def alpha_beta_from_three_coordinates(f, c1, c2, c3):
    """
    Retourne l'angle entre l'objet 1 et 2 (alpha), puis l'angle entre l'objet 2 et 3 (beta). 
    Les arguments c1, c2, c3 sont la position en x de chaque objet dans l'image. f est la
    longueur focale. Les angles retournes sont en degrees
    """
    position_of_optical_axis = 1920 // 2  # valeur en px

    alpha = ...  # TODO calculer l'angle entre 1 et 2
    beta = ...  # TODO calculer l'angle entre 2 et 3

    return alpha, beta


def circle_from_pts_and_angle(pts, angle):
    """
    Construit un cercle à partir de deux points de ce cercle et de l'angle
    entre ces deux points vu par un objet qui est aussi sur le cercle. pts doit
    être un tuple de points. Le point le plus à gauche doit toujours être donné
    en premier.
    """
    (p1, p2) = pts

    q = distance(p1, p2)
    m = (p1 - p2) / 2. + p2  # Point milieu entre les deux points connus
    v = np.array([[0, -1], [1, 0]]).dot(m - p2)  # Vecteur perpendiculaire à la droite reliant p1 et p2

    l = (q / 2) / np.tan(np.radians(angle))  # Distance entre le points milieu et le centre du cercle

    v = (v / lin.norm(v)) * l  # Ajustement de la longueur du vecteur

    c = m + v  # Centre du cercle
    r = np.fabs((q / 2.) / np.sin(np.radians(angle)))  # Rayon du cercle

    return (c.transpose()[0], r)

In [ ]:
# TODO - mX représente la coordonnée en pixel de pX dans l'image
m1 = 200.
m2 = 300.
m3 = 500.

p1 = np.array([[-1.], [1.]])
p2 = np.array([[0.], [0.]])
p3 = np.array([[1.], [0.]])

(alpha, beta) = alpha_beta_from_three_coordinates(f, m1, m2, m3)

(c1, r1) = circle_from_pts_and_angle((p1, p2), alpha)
(c2, r2) = circle_from_pts_and_angle((p2, p3), beta)

print('Premier cercle centré en {} avec un rayon de {}'.format(c1, r1))
print('Second cercle centré en {} avec un rayon de {}'.format(c2, r2))

In [ ]:
circle1 = plt.Circle(c1, r1, edgecolor='r', facecolor='none', linewidth=2.0)
circle2 = plt.Circle(c2, r2, edgecolor='b', facecolor='none', linewidth=2.0)

fig, ax = (plt.gcf(), plt.gca())
ax.add_artist(circle1)
ax.add_artist(circle2)

points = np.array([p1, p2, p3])
plt.scatter(points[:, 0], points[:, 1], s=100, marker='s', color='k')

ax.set_xticks(np.arange(-10, 10, 1))
ax.set_yticks(np.arange(-10, 10, 1))
ax.set_xlim([-5, 5])
ax.set_ylim([-5, 5])
ax.set_aspect('equal', 'datalim')
ax.grid()

plt.show()

## Partie 4 - Effet du bruit sur la localisation

Les fonctions qui suivent servent à montrer l'effet du bruit des mesures sur la position estimée du robot. Ici nos mesures sont les positions en x des repères dans l'image. La boucle qui suit va ajouter un bruit aléatoire (selon une distribution normale) sur vos mesures. Ensuite, elle recalcule la position du robot comme dans la partie précédente. Finalement, on trace un graphique de toutes les positions estimées.

In [ ]:

from robmob.geometry import circle_intersection

measures = np.array([m1, m2, m3])

MEAN_NOISE = 0.0
NOISE_STD_DEV = 5.0

# Construire une liste d'hypothèses sur la position du robot.
intersection_list = np.empty((0, 2))
for i in range(100):
    # On ajoute un bruit gaussien aux mesures
    noisy_measures = measures + np.random.normal(MEAN_NOISE, NOISE_STD_DEV, (3))

    (alpha, beta) = alpha_beta_from_three_coordinates(f, *noisy_measures)

    noisy_c1 = circle_from_pts_and_angle((p1, p2), alpha)
    noisy_c2 = circle_from_pts_and_angle((p2, p3), beta)

    intersections = circle_intersection((noisy_c1[0][0], noisy_c1[0][1], noisy_c1[1]),
                                        (noisy_c2[0][0], noisy_c2[0][1], noisy_c2[1]))

    new_rows = np.array([intersections[0], intersections[1]])
    intersection_list = np.concatenate([intersection_list, new_rows], axis=0)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

circle1 = plt.Circle(c1, r1, edgecolor='r', facecolor='none', linewidth=2.0)
circle2 = plt.Circle(c2, r2, edgecolor='b', facecolor='none', linewidth=2.0)

ax.add_patch(circle1)
ax.add_patch(circle2)

ax.scatter(intersection_list[:, 0], intersection_list[:, 1])

ax.set_xticks(np.arange(-10, 10, 1))
ax.set_yticks(np.arange(-10, 10, 1))
ax.set_xlim([-5, 5])
ax.set_ylim([-5, 5])
ax.set_aspect('equal', 'datalim')
ax.grid()

Répétez l'expérience avec différents niveaux de bruits, que remarquez-vous?

Que ce passe-t'il si vous éloignés les points de repères?